```json
            { 
                "tag": "q_0000_0001", 
                "type": "question", 
                "description": { 
                    "number": "", 
                    "question": "", 
                    "options": [], 
                    "options": null, 
                    "answer": "", 
                    "explanation": "" 
                    }, 
                "caption": [ 
                    "" 
                    ], 
                "file_path": null, 
                "bbox": null 
            }, 
{ 
    "tag": "note_0000_0001", 
    "type": "footnote", 
    "description": "1 이러한 계약을 법적으로는 금전소비대차계약이라고 한다. 차용증서를 영어로는 I owe you.의 소리를 따라 IOU 라고 한다.", 
    "caption": null, 
    "file_path": null, 
    "bbox": null 
}, 
{ 
    "tag": "tb_0000_0001", 
    "type": "table", 
    "description": "", 
    "caption": [], 
    "file_path": "??????/crop/tb_0000_0001.png", 
    "bbox": null 
}, 
{ 
    "tag": "img_0000_0001", 
    "type": "image", 
    "description": null, 
    "caption": [], 
    "file_path": "?????/crop/img_0000_0001.png", 
    "bbox": null 
}
```

### 도서 정보 추출

In [ ]:
# import tools.ProcessFiles as pf
# excel = pf.get_excel_data(3) # i차 분석
# excel

### Lv2 파일명 변경, Lv3/4/5 폴더 분류

In [1]:
import pandas as pd
import os

base_path = "/Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar"
analysis = {1:'1차 분석', 2:'2차 분석', 3: '3차 분석'}
buy = {1:'1차 구매', 2:'2차 구매', 3: '3차 구매'}
i = 1

excel_analy = pd.read_excel(os.path.join(base_path, 'book_list_ALL.xlsx'), sheet_name=analysis[i], header=3)[['관리번호_old', '관리번호', 'ISBN', '도서명', '분류']]

In [ ]:
import shutil
from tqdm import tqdm

zip_path = "/Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/zip/250826_1차_원천코퍼스_보완사항반영"
original_path = f"/Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/{i}C"

error_list = []

for fn, tn, isbn, lv in tqdm(zip(excel_analy['관리번호_old'], excel_analy['관리번호'], excel_analy['ISBN'], excel_analy['분류'])):
   fn = str(fn)
   if '/' in lv: 
      lv = lv.replace("/", '_')
   
   if 'Lv' not in lv:
      continue

   # os.system(f"cd /Users/yejin/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/1C/Lv2 && mv {isbn}.json {tn}.json")
   if lv == 'Lv2':
      # os.system(f"cd /Users/yejin/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/1C/Lv2 && mkdir {tn} && mv {tn}.json {tn}/")
      continue
   else:
      os.system(f"cp {zip_path}/{fn}.zip {original_path}/{lv}/{tn}.zip && cd {original_path}/{lv} && mkdir {tn} && unzip {tn}.zip -d {tn} && find . -type f -name '*{fn}*'| sed -e 'p' -e 's/{fn}/{tn}/g' | xargs -n 2 mv")

      file_path = f"{original_path}/{lv}/{tn}/{tn}.json"
      try:
         with open(file_path, 'r', encoding='utf-8') as f:
               content = f.read()
         
         # Replace the old filename with new filename
         updated_content = content.replace(str(fn), str(tn))
         
         with open(file_path, 'w', encoding='utf-8') as f:
               f.write(updated_content)
               
         print(f"Updated {fn} to {tn} in {file_path}")
      except Exception as e:
         print(f"Error processing {fn} to {tn}: {e}")
         error_list.append((fn, tn, lv))
         # 이름 바꾸기
         # os.system(f"cd {original_path}/{lv} && mkdir {tn} && unzip {tn}.zip -d {tn} && find . -type f -name '*{fn}*'| sed -e 'p' -e 's/{fn}/{tn}/g' | xargs -n 2 mv")

In [6]:
import shutil
from tqdm import tqdm

zip_path = "/Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/zip/250826_1차_원천코퍼스_보완사항반영"
original_path = f"/Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/{i}C"

error_list = []

for fn, tn, isbn, lv in tqdm(zip(excel_analy['관리번호_old'], excel_analy['관리번호'], excel_analy['ISBN'], excel_analy['분류'])):
   fn = str(fn)
   if '/' in lv: 
      lv = lv.replace("/", '_')
   
   if 'Lv' not in lv:
      continue

   # os.system(f"cd /Users/yejin/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/1C/Lv2 && mv {isbn}.json {tn}.json")
   if lv == 'Lv2':
      # os.system(f"cd /Users/yejin/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/1C/Lv2 && mkdir {tn} && mv {tn}.json {tn}/")
      continue
   else:
    #   os.system(f"cp {zip_path}/{fn}.zip {original_path}/{lv}/{tn}.zip && cd {original_path}/{lv} && mkdir {tn} && unzip {tn}.zip -d {tn} && find . -type f -name '*{fn}*'| sed -e 'p' -e 's/{fn}/{tn}/g' | xargs -n 2 mv")
      os.system(f"cd {original_path}/{lv}/{tn} && find . -type f -name '*{fn}*'| sed -e 'p' -e 's/{fn}/{tn}/g' | xargs -n 2 mv")

      file_path = f"{original_path}/{lv}/{tn}/{tn}.json"
      try:
         with open(file_path, 'r', encoding='utf-8') as f:
               content = f.read()
         
         # Replace the old filename with new filename
         updated_content = content.replace(str(fn), str(tn))
         
         with open(file_path, 'w', encoding='utf-8') as f:
               f.write(updated_content)
               
         print(f"Updated {fn} to {tn} in {file_path}")
      except Exception as e:
         print(f"Error processing {fn} to {tn}: {e}")
         error_list.append((fn, tn, lv))
         # 이름 바꾸기
         # os.system(f"cd {original_path}/{lv} && mkdir {tn} && unzip {tn}.zip -d {tn} && find . -type f -name '*{fn}*'| sed -e 'p' -e 's/{fn}/{tn}/g' | xargs -n 2 mv")

16it [00:00, 75.41it/s]

Updated 225933255 to SS0319 in /Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/1C/Lv3_4/SS0319/SS0319.json
Updated 239373741 to SS0320 in /Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/1C/Lv3_4/SS0320/SS0320.json
Updated 243228629 to SS0321 in /Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/1C/Lv3_4/SS0321/SS0321.json
Updated 250217022 to SS0323 in /Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/1C/Lv3_4/SS0323/SS0323.json
Updated 254160217 to SS0324 in /Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/1C/Lv3_4/SS0324/SS0324.json
Updated 255040895 to SS0325 in /Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/1C/Lv3_4/SS0325/SS0325.json
Updated 269377719 to SS0326 in /Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/1C/Lv3_4/SS0326/SS0326.json
Updated 27135

35it [00:00, 84.03it/s]

Updated 303560744 to SS0335 in /Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/1C/Lv3_4/SS0335/SS0335.json
Updated 306937890 to SS0337 in /Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/1C/Lv3_4/SS0337/SS0337.json
Updated 308423311 to SS0340 in /Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/1C/Lv3_4/SS0340/SS0340.json
Updated 308908783 to SS0341 in /Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/1C/Lv3_4/SS0341/SS0341.json
Updated 310847711 to SS0343 in /Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/1C/Lv3_4/SS0343/SS0343.json
Updated 314844344 to SS0344 in /Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/1C/Lv3_4/SS0344/SS0344.json
Updated 316007919 to SS0346 in /Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/1C/Lv3_4/SS0346/SS0346.json
Updated 31750

46it [00:25,  1.16it/s]

...

Updated 351876680 to SS0406 in /Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/1C/Lv5/SS0406/SS0406.json
Updated 353302817 to SS0407 in /Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/1C/Lv3_4/SS0407/SS0407.json
Updated 353738691 to SS0408 in /Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/1C/Lv5/SS0408/SS0408.json
Updated 356379574 to SS0409 in /Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/1C/Lv3_4/SS0409/SS0409.json
Updated 356500908 to SS0410 in /Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/1C/Lv5/SS0410/SS0410.json
Updated 357094267 to SS0411 in /Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/1C/Lv5/SS0411/SS0411.json
Updated 357314428 to SS0412 in /Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/1C/Lv5/SS0412/SS0412.json
Updated 359837661 to SS

104it [00:39,  1.96it/s]

Updated 362874088 to SS0422 in /Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/1C/Lv5/SS0422/SS0422.json
Updated 364690338 to SS0423 in /Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/1C/Lv5/SS0423/SS0423.json


106it [00:43,  2.43it/s]

Updated 346764368 to SS0424 in /Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/ORIGINAL/1C/Lv3_4/SS0424/SS0424.json


In [ ]:
import os

# 업로드
for ss in file:
    print(ss)
    os.system(f"scp -r /Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/FINAL/3C/Lv3_4/{ss} yjmoon@oci004:/home/yjmoon/FINAL/{ss}")

In [ ]:
for fn, tn, lv in zip(excel_analy['관리번호'], excel_analy['관리번호_new'], excel_analy['분류']):
    fn = str(fn)
    if '/' in lv: 
       lv = lv.replace("/", '_')
#     print(fn, tn, lv)
    
    dir = os.path.join(pf.FINAL_DATA_PATH, pf.CYCLE_PATH[1], lv)
    os.system(f'cd {dir} && mv {fn}_low {tn}_low && cd {tn} &&'+f' for file in *{fn}*; do mv "$file" "$'+'{file//'+fn+'/'+tn+'}"; done')
    print(os.path.join(dir, fn))
    # break

In [ ]:
import tools.ProcessFiles as pf

pf.move_jsons(3, pf.ORIGINAL_DATA_PATH)

---

### 개별 처리
- 불필요 페이지 제거
- 오타
- 페이지 머리말/꼬리말

## 전체 돌리기

In [ ]:
import os, json
from typing import List, Dict, Any
import tools.ProcessFiles as pf
import tools.ProcessLv2 as pl2

json_files = pf.get_filelist(1)
json_files

In [ ]:
# import os, json, shutil
# import tools.ProcessFiles as pf

# i = 3
# # 폴더 만들어 옮기기
# excel = pf.get_excel_data(i)
# FINAL_DATA_PATH = os.path.join("/Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter", '3C')
# ORIGINAL_DATA_PATH = os.path.join("/Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/FINAL/", '3C')

# json_files = pf.get_filelist(i)

# for j in json_files:
#     new_path = j.replace(ORIGINAL_DATA_PATH, FINAL_DATA_PATH)
#     try:
#         shutil.copy(j, new_path)
#     except FileNotFoundError:
#         os.makedirs(os.path.dirname(new_path), exist_ok=True)
#         shutil.copy(j, new_path)

In [ ]:
import json
import tools.ProcessLv2 as pl2

# for j in json_files:
if True:
    j = '/Users/jinym/Desktop/Desktop_AICenter✨/SFAIcenter/data/FINAL/2C/Lv2/SS0017_low/SS0017.json'
    INPUT_PATH = j
    BACKUP_PATH = INPUT_PATH + ".bak"

    with open(INPUT_PATH, "r", encoding="utf-8") as f:
        origin = json.load(f)

    with open(BACKUP_PATH, "w", encoding="utf-8") as f:
        json.dump(origin, f, ensure_ascii=False, indent=4)

    # id = excel[excel['ISBN'] == int(origin.get('file_id'))].index[0]
    # break
    # id = os.path.splitext(os.path.basename(j))[0]
    # try:
    #     new = {
    #         'file_id': id,
    #         'ISBN': str(excel.loc[id, 'ISBN']),
    #         'title': excel.loc[id, '도서명'],
    #         'cat1_domain': excel.loc[id, '코퍼스 1분류'],
    #         'cat2_sub': excel.loc[id, '코퍼스 2분류'],
    #         'cat3_specific': excel.loc[id, '비고'],
    #         'pub_date': str(excel.loc[id, '출판일'])[:10],
    #         # 'contents': [],
    #         'contents': origin['contents']
    #     }
    # except:
    #     new = origin
    
    new = pl2.fill_chapter(origin)
    # new = pl2.merge_paragraphs(origin)
    # new = pl2.erase_page(origin, 1)
    # new = pl2.format_change(3,origin)
    # new = origin

    # for i in range(len(new['contents'])):
        # contents = new['contents'][i]
        # # c = extract_qna(contents)
        # page_contents = pl2.remove_enter(contents['page_contents'])
        # contents['page_contents'] = page_contents
        # # c = contents

    with open(INPUT_PATH.replace(".json", ".json"), "w", encoding="utf-8") as f:
    # with open(INPUT_PATH.replace(".json", "_new.json"), "w", encoding="utf-8") as f:
        json.dump(new, f, ensure_ascii=False, indent=4)